# The TinyALU TLM Testbench

In [1]:
from pyuvm import *
from tb_pkg import *
import random

class Driver(uvm_component):
    def build_phase(self):
        self.dut = self.cdb_get("DUT")
    
    def run_phase(self):
        self.raise_objection()
        for _ in range(5):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(Ops))
            cmd = AluCommand("cmd", A, B, op)
            self.dut.send_op(cmd)
        time.sleep(0.6) # Wait for last operation to complete
        self.drop_objection()        

class Coverage(uvm_subscriber):
    
    def end_of_elaboration_phase(self):
        self.cvg = set()
    
    def write(self, cmd):
        self.cvg.add(cmd.op)

    def check_phase(self):
        if len(set(Ops) - self.cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-self.cvg}")

class Scoreboard(uvm_component):  

    def build_phase(self):
        self.cmd_fifo = uvm_tlm_analysis_fifo("cmd_fifo", self)
        self.rslt_fifo = uvm_tlm_analysis_fifo("rslt_fifo", self)
        self.cmd_get_port = uvm_get_port("cmd_get_port", self)    
        self.rslt_get_port = uvm_get_port("rslt_get_port", self) 
        self.cmd_export = self.cmd_fifo.analysis_export
        self.rslt_export = self.rslt_fifo.analysis_export

    def connect_phase(self):
        self.cmd_get_port.connect(self.cmd_fifo.get_export)
        self.rslt_get_port.connect(self.rslt_fifo.get_export)

    def run_phase(self):
        while True:
            rtxn = self.rslt_get_port.get()
            cmd = self.cmd_get_port.get()  
            actual_result = rtxn.result
            predicted_result = TinyAluTlm.alu_op(cmd.A, cmd.B, cmd.op)
            if predicted_result == actual_result:
                self.logger.info(f"PASSED: 0x{cmd.A:02x} {cmd.op.name} 0x{cmd.B:02x} ="
                                 f" 0x{actual_result:04x}")
            else:
                self.logger.error(f"FAILED: 0x{cmd.A:02x} {cmd.op.name} 0x{cmd.B:02x} "
                                  f"= 0x{actual_result:04x} expected 0x{predicted_result:04x}")     

                
class Monitor(uvm_component):
    def __init__(self, name, parent, method_name):
        super().__init__(name, parent)
        self.method_name = method_name
    
    def build_phase(self):
        self.dut = self.cdb_get("DUT")
        self.ap = uvm_analysis_port("ap", self)

    def run_phase(self):
        while True:
            get_method = getattr(self.dut, self.method_name)
            datum = get_method()
            self.ap.write(datum)    
            
class AluEnv(uvm_env):

    def build_phase(self):
        self.cmd_mon = Monitor("cmd_mon", self, "get_cmd")
        self.rslt_mon = Monitor("rslt_mon", self, "get_result")
        self.scoreboard = Scoreboard("scoreboard", self)
        self.coverage = Coverage("coverage", self)
        self.driver = Driver("driver", self)
        
    def connect_phase(self):
        self.cmd_mon.ap.connect(self.scoreboard.cmd_export)
        self.cmd_mon.ap.connect(self.coverage)
        self.rslt_mon.ap.connect(self.scoreboard.rslt_export)
        

class AluTest(uvm_test):
    def build_phase(self):
        self.dut = TinyAluTlm("dut", self)
        self.cdb_set("DUT", self.dut)
        self.env = AluEnv("env", self)

uvm_root().run_test("AluTest")
        

INFO: <ipython-input-1-099ceffeb103>(54)[uvm_test_top.env.scoreboard]: PASSED: 0xdf AND 0xc7 = 0x00c7
INFO: <ipython-input-1-099ceffeb103>(54)[uvm_test_top.env.scoreboard]: PASSED: 0x79 XOR 0x60 = 0x0019
INFO: <ipython-input-1-099ceffeb103>(54)[uvm_test_top.env.scoreboard]: PASSED: 0xd8 XOR 0xbe = 0x0066
INFO: <ipython-input-1-099ceffeb103>(54)[uvm_test_top.env.scoreboard]: PASSED: 0x60 NOP 0x7d = 0x0060
INFO: <ipython-input-1-099ceffeb103>(54)[uvm_test_top.env.scoreboard]: PASSED: 0x3f XOR 0xc8 = 0x00f7
ERROR: <ipython-input-1-099ceffeb103>(30)[uvm_test_top.env.coverage]: Functional coverage error. Missed: {<Ops.ADD: 1>, <Ops.MUL: 4>}
